In [1]:
def reverse_tuple(tuples):
    new_tup = tuples[::-1]
    return new_tup

def read_maps(size, n_sp, n_tar, id):
    map_start = []
    map_end = []
    T = []
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\start\\graph_start_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_start:
        T_ub = int(file_start.readline())
        for line in file_start.readlines():
            # map(function/type, iterable_object)，把測資用 map 轉整數，然後轉成 list
            map_start += [list(map(int, line.split(',')))]
    
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\end\\graph_end_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_end:
        T_ub = int(file_end.readline())
        for line in file_end.readlines():
            map_end += [list(map(int, line.split(',')))]

    T = [i+1 for i in range(T_ub)]
    return T_ub, T, map_start, map_end

def set_start_end(size, map_grid, map_start, map_end, N, R):
    I = []
    P = []
    S_ri = {r:{i:0 for i in N} for r in R}
    F_ri = {r:{i:0 for i in N} for r in R}
    for i in range(size[0]):
        for j in range(size[1]):
            # I
            if map_start[i][j] == -1:
                I.append((0, map_grid[i][j]))
            elif map_start[i][j] != 0:
                I.append((map_start[i][j], map_grid[i][j]))
            # P
            if map_end[i][j] == -1:
                P.append((0, map_grid[i][j]))
            elif map_end[i][j] != 0:
                P.append((map_end[i][j], map_grid[i][j]))
    I = sorted(I)
    P = sorted(P)
    for (r, i) in I:
        S_ri[r][i] = 1
    for (r, i) in P:
        F_ri[r][i] = 1

    return I, P, S_ri, F_ri

def set_arcs(size, map_grid, N, R):
    A = []
    In_ri = {i: {} for i in R}
    Out_ri = {i: {} for i in R}
    Vi = {i: [] for i in N}
    Hi = {i: [] for i in N}
    
    for i in range(size[0]):
        for j in range(size[1]):
            node = map_grid[i][j]
            neighbors = [(node,node)]
            if j < size[1] - 1:
                right = map_grid[i][j+1]
                neighbors.append((node, right))
                Hi[node].append((node, right))
            if j > 0:
                left = map_grid[i][j-1]
                neighbors.append((node, left))
                Hi[node].append((node, left))
            if i < size[0] - 1:
                down = map_grid[i+1][j]
                neighbors.append((node, down))
                Vi[node].append((node, down))
            if i > 0:
                up = map_grid[i-1][j]
                neighbors.append((node, up))
                Vi[node].append((node, up))
    
            A.extend(neighbors)
            Out_ri[node] = neighbors
            In_ri = Out_ri.copy()
    
    return A, In_ri, Out_ri, Vi, Hi
    
def set_parameter(F_ri):
    M = 10000000
    epsilon = 0.001
    Num = [0] * len(F_ri)
    for r in range(len(Num)):
        for i in F_ri[r].values():
            if i == 1: Num[r] += 1
    return M, epsilon, Num

def preprocess(size, n_sp, n_tar, id):
    map_grid = [[i*size[1] + j + 1 for j in range(size[1])] for i in range(size[0])]
    N = [i for i in range(1, size[0]*size[1]+1)]
    R = [i for i in range(n_tar+1)]   # 其中 0 代表 non-target 推盤
    
    T_ub, T, map_s, map_e = read_maps(size, n_sp, n_tar, id)
    # 與 kiva 相比
    # map_s = [
    #     [-1, -1, 2],
    #     [3, -1, 0],
    #     [-1, 0, 1],
    # ]
    # map_e = [
    #     [-1, -1, 1],
    #     [2, -1, 3],
    #     [-1, 0, 0],
    # ]
    
    # 與 DQN 相比
    # map_s = [
    #      [1, 2, 3, 4],
    #      [5, 6, 7, 8],
    #      [9, 10, 11, 0],  # 0 代表空格
    #      [13, 14, 15, 12]]
    
    # map_e = [[1, 4, 11, 8],
    #         [6, 3, 7, 12],
    #         [5, 2, 0, 14],
    #         [13, 9, 10, 15]]

    # 無解
    # map_s = [[3,1,2],
    #          [6,4,5],
    #          [8,7,0]]
    # map_e = [[1,2,3],
    #          [4,5,6],
    #          [8,7,0]]

    # 與 dp 相比
    # map_s = [
    #     [2, 3, 1],
    #     [4, 5, -1],
    #     [0, -1, 0]
    # ]
    # map_e = [
    #     [0, -1, 0],
    #     [5, -1, 3],
    #     [2, 4, 1]
    # ]
    A, In_ri, Out_ri, Vi, Hi = set_arcs(size, map_grid, N, R)
    I, P, S_ri, F_ri = set_start_end(size, map_grid, map_s, map_e, N, R)
    M, epsilon, Num = set_parameter(F_ri)

    return map_grid, map_s, map_e, T_ub, T, N, R, A, In_ri, Out_ri, Vi, Hi, I, P, S_ri, F_ri, M, epsilon, Num
    
# map_grid, map_s, map_e, T_ub, T, N, R, A, In_ri, Out_ri, Vi, Hi, I, P, S_ri, F_ri, M, epsilon, Num = preprocess((3,3),2,4,4)
# print(map_grid, map_s, map_e, T_ub, T, N, R, A, In_ri, Out_ri, Vi, Hi, I, P, S_ri, F_ri, M, epsilon, Num)